In [6]:
import sys
sys.path.append("../")
sys.path.append("../pympc/")
sys.path.append("../pympc/control/hscc/")
import subprocess
import numpy as np

In [13]:
N = 25

solution = dict()
solution_file = "./solutions/N25_h0.05_seed2.mps.solution"
with open(solution_file, 'r') as in_file:
    for line in in_file.readlines():
        variable, value = line.split()
        solution[variable] = float(value)

N = 30 if "N30" in solution_file else 25
x = np.array([np.array([solution[f'x@{t}[{k}]'] for k in range(10)]) for t in range(N+1)])
assert(len(x) == N + 1)

The following cells use MeshCat (available at https://github.com/rdeits/meshcat-python) to generate an animation of the optimal trajectory of the ball and the paddle.

In [14]:
import meshcat
import meshcat.transformations as tf
from meshcat.geometry import Box, Sphere, Cylinder, MeshLambertMaterial
from meshcat.animation import Animation

In [15]:
# initialize visualizer
vis = meshcat.Visualizer()
vis.open()

You can open the visualizer by visiting the following URL:
http://127.0.0.1:7002/static/


<Visualizer using: <meshcat.visualizer.ViewerWindow object at 0x7fb0fc6fc640> at path: <meshcat.path.Path object at 0x7fb0fc6fce50>>

In [16]:
# geometric parameters of the ball and the paddle
import numeric_parameters as params
tickness = .01
depth = .3

# colors of the scene
red = 0xff5555
blue = 0x5555ff
green = 0x55ff55
grey = 0xffffff

vis['ball_right'].set_object(
    Sphere(params.r),
    MeshLambertMaterial(color=blue)
)
vis['ball_left'].set_object(
    Sphere(params.r),
    MeshLambertMaterial(color=green)
)
vis['floor'].set_object(
    Box([depth, params.l*2., tickness]),
    MeshLambertMaterial(color=red)
)
vis['ceiling'].set_object(
    Box([depth, params.l*2., tickness]),
    MeshLambertMaterial(color=grey)
)

In [17]:
x = list(x)
x = np.array([x[0]] * 5 + x)

In [20]:
# initialize animation
anim = Animation()

# animate, e.g., the solution with infinity norm (and convex-hull method -- irrelevant)

for t, xt in enumerate(x):
    with anim.at_frame(vis, t*params.h*80) as frame: # 30 frames per second to get real time
        frame['ball_right'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+params.r]).dot(
                tf.rotation_matrix(xt[2], [1.,0.,0.]).dot(
                    tf.translation_matrix([0, -0.001, 0])
                )
            )
        )
        frame['ball_left'].set_transform(
            tf.translation_matrix([0, xt[0], xt[1]+params.r]).dot(
                tf.rotation_matrix(xt[2], [1.,0.,0.]).dot(
                    tf.translation_matrix([0, 0.001, 0])
                )
            )
        )
        frame['floor'].set_transform(
            tf.translation_matrix([0, xt[3], xt[4]-tickness/2.])
        )
        frame['ceiling'].set_transform(
            tf.translation_matrix([0, 0, params.d+tickness/2.])
        )

# visualize result
vis.set_animation(anim)